# NOTES EDGES AND LINES DETECTION

# 1. Kernel Convolutions

[How blurs and filters work](https://www.youtube.com/watch?v=C_zFhWdM4ic)

[Image Kernels Explained](http://setosa.io/ev/image-kernels/)

The core of Gaussian blurs, edge detection, mean blurs, etc.

For every pixel in the image, we put our kernel on top of the image so that the pixel is in the middle. Then we multiply each value in the kernel by overlaying values on the image (around the pixel), and we normalize by dividing by the number of pixels in the kernel.

Comments:
- It feels like an average convolution in case all the values in the kernel are the same.
- Otherwise it's a weigthed average.
- If all the values in the kernel are 1, we have a `mean blur`. If you use Photoshop or any other image processing product, and select `blur`, then `mean`, what happens is that you get a `mean blur`. Or a kernel convolution with a kernel of all values equal to 1.
- **The pixel has to be in the CENTER of the kernel, so the kernel must have a center value, so the kernel size must have an odd value: 3, 5, 7, etc**


After computing the kernel comvolution (and getting the average), we get a single value; then we overwrite the value of that pixel with the output of the kernel convolution.

***Note:*** We should output the new pixel value to a different image, because if we overwrite the pixels as we go, it's going to mess with the maths as we go down.

We use blurs to maintain the quality of an image and remove the noise.


### Gaussian Blur (Kernel)


Gaussian blurs are more edge-preserving than mean blurs.

This is an example of a Gaussian kernel. Typically, the kernel values will be drawn from a normal distribution, and be floating point numbers. This is a simplification.

<table>
    <tr>
        <td>1</td>
        <td>2</td>
        <td>1</td>
    </tr>
     <tr>
         <td>2</td>
         <td>4</td>
         <td>2</td>
    </tr>
     <tr>
         <td>1</td>
         <td>2</td>
         <td>1</td>  
    </tr>     
    
</table>

Here, the center weight of the kernel is the strongest, and whichever pixel goes further from the center receives a smaller weight.

# 2. Edge Detection


## 2.1 Finding Edges with the Sobel operator

[Refrence](https://www.youtube.com/watch?v=uihBwtPIBxM)

Edge detection is a kernel convolution method designed to find regions in an image where we have a sharp change in pixel values. High values indicate a steep change, and low values indicate a shallow change. A very common operator to do this is the ***Sobel*** operator. It's an approximation to the derivative of an image.

**How does it work?**

Sobel and Feldman presented the idea of an "isotropic 3x3 image gradient operator" at a talk at the Stanford AI Lab in 1968. It's a discrete differentiator operator, computing an approximation of the gradient of the image intensity. At each point, the output of the sobel operator is the corresponding gradient vector, or the norm of the gradient vector.

The sobel operator uses two 3x3 kernels which are convolved with the original image. if $\textbf{A}$ is the original image, $\textbf{G}_x$ and $\textbf{G}_y$ are the two images generated by the two Sobel kernels, designed to capture vertical and horizontal derivatives of $\textbf{A}$.

$$
\textbf{G}_x = \begin{bmatrix}
            +1 && 0 && -1 \\
            +2 && 0 && -2 \\
            +1 && 0 && -1
            \end{bmatrix} 
            \ast \textbf{A}   \tag{1}           
$$


And 

$$
\textbf{G}_y = \begin{bmatrix}
            +1 && +2 && +1 \\
            0 && 0 && 0 \\
            -1 && -2 && -1
            \end{bmatrix}
            \ast \textbf{A}   \tag{2}
$$

After the two convolutions, the gradients in each direction are normalized to generate the final image as:

$$ \textbf{G} = \sqrt{\textbf{G}_x^2 + \textbf{G}_y^2} \tag{3} $$

Some notes about Sobel:

- Sobel a grayscale image only operator.
- Sobel very noisy: using Sobel will capture some noisy pixels as edges even though they aren't real edges.
- It's very common to use a Gaussian Kernel convolution (or blurring) before applying the Sobel operator, so that we lose some of the noise.


**What is the angular direction of the edge?**

The Sobel operator generates two gradient edge images in both $x$ and $y$ directions, $\textbf{G}_x$ and $\textbf{G}_y$. Because the final edge intensity is detected by $\textbf{G} = \sqrt{\textbf{G}_x^2 + \textbf{G}_y^2}$, it's the hypothenuse of the angle $\theta$ formed by the edge, for which the tangent is given by $\dfrac{\textbf{G}_y}{\textbf{G}_x}$.

Therefore, we can find the orientation of the edge with:

$$\theta = \text{arctan} ({\dfrac{\textbf{G}_y}{\textbf{G}_x}}) \tag{4}$$

$\textbf{G}$ can be computed using the hypot function and $\text{atan2}$ is the arctangent function with two arguments. The edge direction angle is rounded to one of four angles representing vertical, horizontal and the two diagonals (0°, 45°, 90° and 135°). An edge direction falling in each color region will be set to a specific angle values, for instance θ in [0°, 22.5°] or [157.5°, 180°] maps to 0°.



## 2.2 Canny edge detector

Reference: https://en.wikipedia.org/wiki/Canny_edge_detector

The ***Canny edge detector*** is an edge detection operator that uses multiple stages to detect a wide range of edges in images. It was developed by John F. Canny in 1986.

The Process of Canny edge detection algorithm can be broken down to 5 different steps:

1. Apply ***Gaussian filter*** to smooth the image in order to remove the noise
2. Find the intensity gradients of the image
3. Apply non-maximum suppression to get rid of spurious response to edge detection
4. Apply double threshold to determine potential edges
5. Track edge by ***hysteresis***: Finalize the detection of edges by suppressing all the other edges that are weak and not connected to strong edges.

***Note:*** After step2, we have a Sobel edge detection. Therefore, Canny edge detection includes Sobel steps, and 3 more: non-maximum suppression to get rid of spurious responses, double threshold to detect potential edges, and hysteresis to suppress all other edges that are between the thresholds and are ***not*** connected to strong edges.

**Non-maximum suppression** 

This is an edge thinning technique. This is applied to find the "largest" edge. The suppression algorithm compares pixel values for each pixel, with neighboring pixels across gradient directions, and only keeps the largest pixels that indicate highest gradient.

The algorithm for each pixel in the gradient image is:

1. Compare the edge strength of the current pixel with the edge strength of the pixel in the positive and negative gradient directions.
2. If the edge strength of the current pixel is the largest compared to the other pixels in the mask with the same direction (i.e., the pixel that is pointing in the y-direction, it will be compared to the pixel above and below it in the vertical axis), the value will be preserved. Otherwise, the value will be suppressed.

The sign of the direction is irrelevant.

**Double Threshold**

After the non-maximum suppression technique, the remaining edges provide a more accurate (thin) representation of real edges in an image. We still have edge pixels that might be generated by noise. To get rid of those, we apply a ***double threshold*** that works as follows:

- If an edge pixel’s gradient value is higher than the high threshold value, it is marked as a strong edge pixel. 
- If an edge pixel’s gradient value is smaller than the high threshold value and larger than the low threshold value, it is marked as a weak edge pixel. 
- If an edge pixel's value is smaller than the low threshold value, it will be suppressed. 

The two threshold values are empirically determined and their definition will depend on the content of a given input image.


**Hysteresis thresholding**

Finally, we have identified strong and weak edge pixels. The strong edge pixels should be in the final image; they are strong indicators of edges in the real image. The weak edge pixels could either come from a real edge in the image, or some noise/color variations. To get an accurate result, the weak edges caused by noise or color variations should be removed. Weak edges related to a true edge would typically be ***connected*** to a strong edge. We want to preserve the weak edges that are connected to strong edges and discard those that aren't. This process is called hysteresis 


# 3. Hough Transforms Intuition

Reference: https://www.youtube.com/watch?v=kMK8DjdGtZo
